### The [langchain framework](https://python.langchain.com/docs/get_started/introduction.html) makes it easy to use LLMs as [agents](https://python.langchain.com/docs/modules/agents/) capable of making decisions. Furthermore, these agents can be equipped with a variety of [tools](https://python.langchain.com/docs/modules/agents/tools/) which implement different functionalities. LLM agents can be given access of a combination of such tools. The decision to use a particular tool as part of solving a particular task is based on the language understanding ability of the LLMs. 


### In this post, we will explore the power of Langchain agents and their application in question answering over a text document store. Specifically, we will dive into creating a Jupyter Notebook that demonstrates the integration of Large Language model (LLM) with external text documents and a search engine, enabling seamless information retrieval. We'll also discuss the challenges encountered during the planning phase and how to overcome them. In particular we discuss how to make agents use the tools in a particular order, whenever they decide to use them.

### We start with installing and importing relevant libraries and setting up the openAI and SERP API keys. openAI API keys can be obtained from https://platform.openai.com/account/api-keys and SERP API keys can be obtained after registering at https://serpapi.com/. You can use an open source model like Llama2 instead. Langchain serves the functionality to load it via llama-cpp-python library.

### You can clone the git repo: and run this notebook from within the repo. 

In [ ]:
# if you have cloned the git repo, the requirements.txt file will list all required libraries
!pip install -r requirements.txt

In [64]:
import os
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.agents import load_tools
from langchain import OpenAI, SerpAPIWrapper
from langchain.document_loaders import DirectoryLoader, TextLoader


os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
os.environ["SERPAPI_API_KEY"] = "your-serp-api-key"
llm = OpenAI(model_name="text-davinci-003", temperature = 0)

## Connecting to a Document Store

### Now that you have the OpenAI LLM model ready, let's move on to connecting it to documents and fetching data. To achieve this, we'll make use of Directory document loader available in Langchain. I have some text files stored in another directory in the same git repo. This can be replaced by any document store as a wiki/knowledge base, e.g. Confluence, Google Drive, S3 bucket, Any database, etc. Langchain has hunderds of [connectors](https://python.langchain.com/docs/modules/data_connection/document_loaders/) to third party document storage tools.


### The documents I have used in this document store are notes of some LLM based papers I took while reading them, most of the time copying and pasting the text from those papers.


In [47]:
# Loading the documents from a directory
path = './llm_papers'
loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader)
documents = loader.load()
print(documents[0])

page_content='ChatDoctor Paper\n\n\nfine- tuned model based on 100k real-world patient-physician conversations from an online medical consultation site. \n\nAlso added  autonomous knowledge retrieval capabilities to our ChatDoctor, for ex- ample, Wikipedia or a database as a knowledge brain. \n\nCollected about 100k real doctor-patient conversations from an online medical consultation website HealthCareMagic1. We filtered these data both manually and automatically, removed the identity information of the doctor and patient, and used language tools to correct grammatical errors, and we named this dataset HealthCareMagic-100k.\n\nIn addition, we collected approximately 10k patient-physician conversations from the online medical consultation website iCliniq2 to evaluate the performance of our model.  \n\nFor Q&A in medical scenarios, we collected and compiled a database, partly sampled in Fig. 3, which includes about 700 diseases and their associated symptoms, further medical tests or mea

### Now that we have the documents with us, we do some simple prompting to ask questions of the LLM through Chains.  Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.

### Before that, we also need to do some standard operations - text splitting, embedding creation and indexing them in a vector store.

### Often times your document is too long (like a book) for your LLM. You need to split it up into chunks. Text splitters help with this.There are many ways you could split your text into chunks, experiment with different ones to see which is best for you.


In [49]:
# chunking the documents (splitting text into fixed size chunks)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


### Embeddings create a vector representation of a piece of text. This represents text in semantic or meaning space, so that pieces of text similar in meaning are near to each other in that space. 

### Vector databases provide an easy way to store the embedded text and perform the similarity search. We use Chromadb as the vector store here.

In [50]:
# initialising embeddings
embeddings = OpenAIEmbeddings()

# vectorising documents based on these embeddings
docsearch = Chroma.from_documents(texts, embeddings)

# creating a retrieval chain
llm_papers = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [51]:
# Simple Question answering over Confluence data
query = "What is the REACT model?"
llm_papers.run(query)

' The REACT model is an approach that allows autonomous systems to generate reasoning traces and task-specific actions in an interleaved manner, enabling them to perform dynamic reasoning to create, maintain, and adjust high-level plans for acting while also interacting with the external environments to incorporate additional information into reasoning.'

In [52]:
query = "Which paper works in medical domain?"
llm_papers.run(query)

' The ChatDoctor paper works in the medical domain. It is based on 100k real-world patient-physician conversations from an online medical consultation site and has autonomous knowledge retrieval capabilities. It also collected about 100k real doctor-patient conversations from the online medical consultation website HealthCareMagic1 and approximately 10k patient-physician conversations from the online medical consultation website iCliniq2.'

### But we want more than question answering based on the document stores. We may want to get information which may not be existing in the document store. With langchain, this can be done with tools and agents. Tools can interact with different sources of information and an agent can create the context from document store and decide which tool to use and how to use it with the given context.

### Below I create a custom search tool which is an combination of two tools - one searches the  document store to answer the user query, and the other is the search engine result page (serp) api tool, which I want to use whenever some question cannot be answered with document store data.

### Langchain Tool class takes in three parameters - name of the tool, the functionality of the tool and a description of the tool, which can be useful for an agent to decide when and how to use the tool.

In [53]:


# defining search tool using SerpAPIWrapper
search = SerpAPIWrapper()

search_tool_default = [
    Tool(
        name = "Document Store",
        func = llm_papers.run,
        description = "Use it to lookup information from the document store"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Use this to lookup information from google search engine",
    )]



### Now we create an agent which will use the custom combined tool. We create a React zero shot agent, as we also get to see the reasoning behind sub-tool selection.

In [54]:
# create an agent an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(search_tool_default , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What is the REACT paper in AI about? Who are its authors?")



> Entering new AgentExecutor chain...
 I should look for a paper about AI with the title REACT.
Action: Search
Action Input: "REACT paper AI"
Observation: In this paper, we explore the use of LLMs to generate both reasoning traces and task-specific actions in an interleaved manner, allowing for ...
Thought: I should look for the authors of this paper.
Action: Search
Action Input: "REACT paper AI authors"
Observation: Download a PDF of the paper titled ReAct: Synergizing Reasoning and Acting in Language Models, by Shunyu Yao and 6 other authors. PDF · Other ...
Thought: I now know the final answer.
Final Answer: The REACT paper in AI is titled ReAct: Synergizing Reasoning and Acting in Language Models, and is authored by Shunyu Yao and 6 other authors.

> Finished chain.


'The REACT paper in AI is titled ReAct: Synergizing Reasoning and Acting in Language Models, and is authored by Shunyu Yao and 6 other authors.'

### In the above result, we see that the agent takes the google search action, i.e. uses the SerpAPI tool all the time. We would like it to first lookup for information in the Document store first, before going to google.

### So we specify this in the tool description.

In [55]:
# create new tool with order of tool usage made explicit

search_tool_specify_order_in_dscrptn = [
    Tool(
        name = "Document Store",
        func = llm_papers.run,
        description = "Use it to lookup information from document store. Always used as first tool"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Use this to lookup information from google search engine. \
        Use it only after you have tried using the document store tool.",
    )]



In [56]:
# create an agent which will use the modified tool

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(search_tool_specify_order_in_dscrptn , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# agent.agent.llm_chain.verbose=True
# Now let's test it out!
agent.run("What is the REACT paper in AI about? Who are its authors?")



> Entering new AgentExecutor chain...
 I should look for the paper in the document store first.
Action: Document Store
Action Input: REACT paper in AI
Observation:  The ReAct paper is a research paper in the field of Artificial Intelligence that explores the potential of using large language models to improve the synergy between verbal reasoning and interactive decision making in autonomous systems. It proposes a new approach that allows the model to generate reasoning traces and task-specific actions in an interleaved manner, enabling it to perform dynamic reasoning to create, maintain, and adjust high-level plans for acting while also interacting with the external environments to incorporate additional information into reasoning.
Thought: I should look for the authors of the paper.
Action: Search
Action Input: REACT paper in AI authors
Observation: Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2022). React: Synergizing reasoning and acting in language m

'The authors of the REACT paper in AI are Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2022).'

### This time the agent used the Document Store tool as the first action. Lets use it to answer more questions to see if it is consistent in following the order.

In [57]:
agent.run("Which paper works in the medical domain? Which university are its authors affiliated to?")



> Entering new AgentExecutor chain...
 I need to find a paper and its authors
Action: Search
Action Input: "medical domain paper authors"
Observation: Authors and Contributors in Scientific and Medical Publications · Step 1: Establish an authorship working group early in the trial, · Step 2: ...
Thought: I need to find more specific information
Action: Search
Action Input: "medical domain paper authors university"
Observation: Keywords are important for indexing: they enable your manuscript to be more easily identified and cited. Check the GUIDE FOR AUTHORS for journal requirements.
Thought: I now know the final answer
Final Answer: Authors of papers in the medical domain should check the Guide for Authors of the journal they are submitting to for requirements regarding keywords and affiliations.

> Finished chain.


'Authors of papers in the medical domain should check the Guide for Authors of the journal they are submitting to for requirements regarding keywords and affiliations.'

In [58]:
 agent.run("What is this new Camel paper in LLMs about. Give detailed summary of the paper")



> Entering new AgentExecutor chain...
 I should look for the paper in the document store
Action: Document Store
Action Input: Camel paper in LLMs
Observation:  ReAct prompts LLMs to generate both verbal reasoning traces and actions pertaining to a task in an interleaved manner. This approach enables the model to perform dynamic reasoning, to create, maintain, and adjust high-level plans for acting, while also interacting with the external environments to incorporate additional information into reasoning.
Thought: I should look for more information about the paper
Action: Search
Action Input: Camel paper in LLMs
Observation: This paper explores the potential of building scalable techniques to facilitate autonomous cooperation among communicative agents and ...
Thought: I now know the final answer
Final Answer: This paper explores the potential of building scalable techniques to facilitate autonomous cooperation among communicative agents and proposes a novel approach called ReAct (Rea

'This paper explores the potential of building scalable techniques to facilitate autonomous cooperation among communicative agents and proposes a novel approach called ReAct (Reasoning and Action) that combines a language-based reasoning module with an action-based reasoning module. The proposed approach enables agents to reason and act in an interleaved manner, allowing them to dynamically adjust their plans and incorporate additional information into their reasoning.'

### We see that the agent is not consistent in following the order of tools specified in the tool description. This is because the final prompt which is sent to the model does not explicitly mention this order constraint.

### We need to make changes in the final prompt which is being sent to the model. This can be seen using 'agent.agent.llm_chain.prompt.template'. 

In [59]:
# let see what is final prompt which is being sent to the LLM

print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Document Store: Use it to lookup information from document store. Always used as first tool
Search: Use this to lookup information from google search engine.         Use it only after you have tried using the document store tool.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Document Store, Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### We see that although the tool description mentions that the agent should use Document Store as the first tool, it is still not sticking to this rule. We can instead make this rule explicit in the React method description part of the prompt. Specifically, we can mention it during the Action component of React.

In [60]:
agent.agent.llm_chain.prompt.template = '''
Answer the following questions as best you can. You have access to the following tools:

Document Store: Use it to lookup information from document store. Always used as first tool
Search: Use this to lookup information from google search engine. Use it only after you have tried using the document store tool.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Document Store, Search]. Always look first in Document Store.
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
'''


### Now let's see if making the small and simple change above can make the agent follow our tool order rule better

In [61]:
agent.run("What is the REACT paper in AI about? Who are its authors?")



> Entering new AgentExecutor chain...
I should look for the paper in the Document Store first.
Action: Document Store
Action Input: REACT paper in AI
Observation:  The ReAct paper is a research paper in the field of Artificial Intelligence that explores the potential of using large language models to improve the synergy between verbal reasoning and interactive decision making in autonomous systems. It proposes a new approach that allows the model to generate reasoning traces and task-specific actions in an interleaved manner, enabling it to perform dynamic reasoning to create, maintain, and adjust high-level plans for acting while also interacting with the external environments to incorporate additional information into reasoning.
Thought:The authors of the paper should be listed in the document.
Action: Document Store
Action Input: REACT paper in AI
Observation:  The ReAct paper is a research paper in the field of Artificial Intelligence that explores the potential of using large la

'The authors of the REACT paper in AI are Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2022).'

In [62]:
agent.run("Which paper works in the medical domain? Which university are its authors affiliated to?")



> Entering new AgentExecutor chain...
I need to find out which paper works in the medical domain and which university its authors are affiliated to.
Action: Document Store
Action Input: Paper works in the medical domain
Observation:  Yes, the paper is about a ChatDoctor model that is fine-tuned based on 100k real-world patient-physician conversations from an online medical consultation site. It also has autonomous knowledge retrieval capabilities, such as Wikipedia or a database as a knowledge brain. The paper also collected about 100k real doctor-patient conversations from the online medical consultation website HealthCareMagic1 and approximately 10k patient-physician conversations from the online medical consultation website iCliniq2 to evaluate the performance of the model.
Thought:I need to find out which university the authors are affiliated to.
Action: Search
Action Input: Authors of ChatDoctor paper
Observation: Our proposed ChatDoctor, represents a significant advancement in 

'The paper works in the medical domain and its authors are affiliated to the University of California, Berkeley.'

In [63]:
# lets query about a topic which doesn't exist in the document store.
agent.run("What is this new Camel paper in LLMs about. Give detailed summary of the paper")



> Entering new AgentExecutor chain...
I should first look in the Document Store to see if I can find the paper.
Action: Document Store
Action Input: Camel paper in LLMs
Observation:  ReAct prompts LLMs to generate both verbal reasoning traces and actions pertaining to a task in an interleaved manner. This approach enables the model to perform dynamic reasoning, to create, maintain, and adjust high-level plans for acting, while also interacting with the external environments to incorporate additional information into reasoning.
Thought:I should now search for more information about the paper
Action: Search
Action Input: Summary of Camel paper in LLMs
Observation: This paper explores the potential of building scalable techniques to facilitate autonomous cooperation among communicative agents and provide insight into their "cognitive" processes. To address the challenges of achieving autonomous cooperation, we propose a novel communicative agent framework named role-playing.
Thought:I n

'The Camel paper in LLMs proposes a novel communicative agent framework named role-playing to facilitate autonomous cooperation among communicative agents and provide insight into their "cognitive" processes. The framework enables the model to perform dynamic reasoning, to create, maintain, and adjust high-level plans for acting, while also interacting with the external environments to incorporate additional information into reasoning.'

### We now find the agent is more consistent in following the order of tools - firstly looking up the answer in the document store and only going to google search if it doesn't find anything with the first tool.

### There are still some problems with the final responses we get, especially the university affiliations for the chat doctor paper is wrong. That can be addressed by analysing and processing the response we get from the serp-api. We will see how to parse and validate the output in another blog. This blog was about using multiple tools with an LLM agent and adding some order constraints to the agent.